In [2]:
!pip install numpy

  Using cached numpy-1.21.2-cp38-cp38-win_amd64.whl (14.0 MB)


In [3]:
!pip install transformers

  Using cached transformers-4.10.0-py3-none-any.whl (2.8 MB)
  Using cached huggingface_hub-0.0.16-py3-none-any.whl (50 kB)
  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
  Using cached tokenizers-0.10.3-cp38-cp38-win_amd64.whl (2.0 MB)
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
  Using cached tqdm-4.62.2-py2.py3-none-any.whl (76 kB)
  Using cached sacremoses-0.0.45-py3-none-any.whl (895 kB)
  Using cached idna-3.2-py3-none-any.whl (59 kB)
  Using cached charset_normalizer-2.0.4-py3-none-any.whl (36 kB)
  Using cached urllib3-1.26.6-py2.py3-none-any.whl (138 kB)
  Using cached click-8.0.1-py3-none-any.whl (97 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)


In [5]:
!pip install torch

  Using cached torch-1.9.0-cp38-cp38-win_amd64.whl (222.0 MB)


In [4]:
import numpy as np
import torch
import torch.nn as nn

import transformers

ModuleNotFoundError: No module named 'torch'

In [4]:
state, action, return_to_go = [], [], []

In [5]:
time_step = []

In [6]:
num_actions = 34
num_states = 8 * 5 + 2

## 노가다 작업은 json으로 미리 저장하기

In [7]:
import json

In [14]:
action_input_list = ['Wait', 'W', 'A', 'S', 'D',
                     'WA', 'WD', 'SD', 'SA',
                     'Ws', 'As', 'Ss', 'Ds',
                     'WAs', 'WDs', 'SDs', 'SAs',
                     'Wj', 'Aj', 'Sj', 'Dj',
                     'WAj', 'WDj', 'SDj', 'SAj',
                     'Wsj', 'Asj', 'Ssj', 'Dsj',
                     'WAsj', 'WDsj', 'SDsj', 'SAsj']
num_action_input = len(action_input_list)

In [15]:
action_ids = {}
for id in range(num_action_input):
    action_ids[action_input_list[id]] = id

In [16]:
print(action_ids)

{'Wait': 0, 'W': 1, 'A': 2, 'S': 3, 'D': 4, 'WA': 5, 'WD': 6, 'SD': 7, 'SA': 8, 'Ws': 9, 'As': 10, 'Ss': 11, 'Ds': 12, 'WAs': 13, 'WDs': 14, 'SDs': 15, 'SAs': 16, 'Wj': 17, 'Aj': 18, 'Sj': 19, 'Dj': 20, 'WAj': 21, 'WDj': 22, 'SDj': 23, 'SAj': 24, 'Wsj': 25, 'Asj': 26, 'Ssj': 27, 'Dsj': 28, 'WAsj': 29, 'WDsj': 30, 'SDsj': 31, 'SAsj': 32}


In [23]:
with open('json/action_ids.json', 'w', encoding='utf-8') as f:
    json.dump(action_ids, f, ensure_ascii=False, indent='\t')

In [31]:
state_position_list = ['field', 'wall', 'lowair', 'highair']
state_stamina_list = ['0-19', '20-39', '40-59', '60-79', '80-100']
state_special_list = ['death', 'goal']

num_state_position_list = len(state_position_list)
num_state_stamina_list = len(state_stamina_list)
num_state_special_list = len(state_special_list)

num_state_ids = num_state_position_list * num_state_stamina_list + num_state_special_list

In [32]:
state_ids = {}
state_id_value = 0
for position in state_position_list:
    sub_state = {}
    for stamina in state_stamina_list:
        sub_state[stamina] = state_id_value
        state_id_value += 1
    state_ids[position] = sub_state
    
for special in state_special_list:
    state_ids[special] = state_id_value
    state_id_value += 1

In [33]:
print(state_ids)

{'field': {'0-19': 0, '20-39': 1, '40-59': 2, '60-79': 3, '80-100': 4}, 'wall': {'0-19': 5, '20-39': 6, '40-59': 7, '60-79': 8, '80-100': 9}, 'lowair': {'0-19': 10, '20-39': 11, '40-59': 12, '60-79': 13, '80-100': 14}, 'highair': {'0-19': 15, '20-39': 16, '40-59': 17, '60-79': 18, '80-100': 19}, 'death': 20, 'goal': 21}


In [34]:
with open('json/state_ids.json', 'w', encoding='utf-8') as f:
    json.dump(state_ids, f, ensure_ascii=False, indent='\t')

## 불러와서 시작

In [1]:
import json

In [2]:
with open('json/state_ids.json', 'r') as f:
    state_ids = json.load(f)

In [3]:
state_ids

{'field': {'0-19': 0, '20-39': 1, '40-59': 2, '60-79': 3, '80-100': 4},
 'wall': {'0-19': 5, '20-39': 6, '40-59': 7, '60-79': 8, '80-100': 9},
 'lowair': {'0-19': 10, '20-39': 11, '40-59': 12, '60-79': 13, '80-100': 14},
 'highair': {'0-19': 15, '20-39': 16, '40-59': 17, '60-79': 18, '80-100': 19},
 'death': 20,
 'goal': 21}

In [4]:
with open('json/action_ids.json', 'r') as f:
    action_ids = json.load(f)

In [5]:
len(state_ids['field'])

5

In [6]:
num_states = 0
for state in state_ids.keys():
    if type(state_ids[state]) != int:
        num_states += len(state_ids[state])
    else:
        num_states += state_ids[state]
num_actions = len(action_ids)

print(num_states, num_actions)

61 33


In [7]:
action_ids

{'Wait': 0,
 'W': 1,
 'A': 2,
 'S': 3,
 'D': 4,
 'WA': 5,
 'WD': 6,
 'SD': 7,
 'SA': 8,
 'Ws': 9,
 'As': 10,
 'Ss': 11,
 'Ds': 12,
 'WAs': 13,
 'WDs': 14,
 'SDs': 15,
 'SAs': 16,
 'Wj': 17,
 'Aj': 18,
 'Sj': 19,
 'Dj': 20,
 'WAj': 21,
 'WDj': 22,
 'SDj': 23,
 'SAj': 24,
 'Wsj': 25,
 'Asj': 26,
 'Ssj': 27,
 'Dsj': 28,
 'WAsj': 29,
 'WDsj': 30,
 'SDsj': 31,
 'SAsj': 32}

In [8]:
from py.environment import Environment

ModuleNotFoundError: No module named 'py.environment'